# Lab 4: Capstone - Document Assistant MCP Server

**Objective:** Build a complete MCP server that serves as a Document Assistant, combining tools, resources, and prompts.

**Duration:** ~45 minutes

**What You'll Build:**
- A fully-featured MCP server for document management
- Search tools to find documents by content or metadata
- Resources to expose document contents
- Prompts for common document analysis tasks

**Prerequisites:**
- Completed Labs 1-3
- Understanding of MCP tools, resources, and prompts

## Part 1: Project Overview

The Document Assistant will help users:
1. **Search** documents by keyword or metadata
2. **Read** document contents via resources
3. **Analyze** documents using pre-built prompts

### Architecture

```
Document Assistant MCP Server
├── Tools
│   ├── search_documents(query) - Search by content
│   ├── list_documents() - List all documents
│   └── get_document_stats() - Get statistics
├── Resources
│   ├── doc://{doc_id} - Individual document content
│   └── docs://catalog - Full document catalog
└── Prompts
    ├── summarize - Summarize a document
    ├── compare - Compare two documents
    └── extract-insights - Extract key insights
```

## Part 2: Setting Up the Project

Let's create the project structure and sample documents.

In [ ]:
import os

# Create project directory
os.makedirs("document_assistant", exist_ok=True)
os.makedirs("document_assistant/documents", exist_ok=True)

print("Project structure created!")

In [ ]:
# Create sample documents for testing

documents = {
    "project_proposal.md": '''# Project Proposal: AI Integration

## Executive Summary
This proposal outlines a plan to integrate AI capabilities into our existing product suite.

## Objectives
1. Improve customer experience through intelligent recommendations
2. Automate repetitive tasks to increase efficiency
3. Provide predictive analytics for business decisions

## Timeline
- Phase 1: Research and Planning (Q1)
- Phase 2: Development (Q2-Q3)
- Phase 3: Testing and Deployment (Q4)

## Budget
Estimated total: $500,000
''',
    
    "technical_spec.md": '''# Technical Specification: API Gateway

## Overview
The API Gateway will serve as the central entry point for all microservices.

## Requirements
- Handle 10,000 requests per second
- Support OAuth 2.0 authentication
- Implement rate limiting
- Provide request/response logging

## Architecture
```
Client -> Load Balancer -> API Gateway -> Microservices
```

## Technologies
- Language: Go
- Framework: Gin
- Database: PostgreSQL
- Cache: Redis
''',
    
    "meeting_notes.md": '''# Meeting Notes: Q4 Planning

**Date:** October 15, 2024
**Attendees:** Alice, Bob, Carol, David

## Discussion Points

### Budget Review
- Current spending is 5% under budget
- Proposal to reallocate funds to AI initiative

### Hiring Plans
- Need 3 senior engineers
- 1 product manager opening

### Action Items
- [ ] Alice: Finalize budget proposal
- [ ] Bob: Post job listings
- [ ] Carol: Schedule follow-up meeting

## Next Meeting
October 22, 2024 at 2:00 PM
''',
    
    "security_policy.md": '''# Security Policy

## Purpose
This document outlines security requirements for all company systems.

## Password Requirements
- Minimum 12 characters
- Must include uppercase, lowercase, numbers, and symbols
- Changed every 90 days

## Data Classification
1. **Public**: Marketing materials, public documentation
2. **Internal**: Employee communications, internal processes
3. **Confidential**: Customer data, financial records
4. **Restricted**: Security credentials, encryption keys

## Incident Response
1. Identify and contain the threat
2. Notify security team immediately
3. Document all actions taken
4. Conduct post-incident review
'''
}

# Save documents
for filename, content in documents.items():
    filepath = f"document_assistant/documents/{filename}"
    with open(filepath, "w") as f:
        f.write(content)
    print(f"Created: {filepath}")

print(f"\nCreated {len(documents)} sample documents")

## Part 3: Building the Server - Step by Step

Let's build the Document Assistant incrementally.

### Step 1: Basic Server Structure

First, create the server with document loading capabilities.

In [ ]:
step1_code = '''
"""Document Assistant MCP Server - Step 1: Basic Structure"""

import os
from pathlib import Path
from datetime import datetime
from mcp.server.fastmcp import FastMCP

# Configuration
DOCUMENTS_DIR = Path(__file__).parent / "documents"

# Create the MCP server
mcp = FastMCP(
    "Document Assistant",
    description="An MCP server for document management and analysis"
)

# Document storage (loaded at startup)
document_cache = {}

def load_documents():
    """Load all documents from the documents directory."""
    global document_cache
    document_cache = {}
    
    if not DOCUMENTS_DIR.exists():
        return
    
    for filepath in DOCUMENTS_DIR.glob("*.md"):
        doc_id = filepath.stem  # filename without extension
        stat = filepath.stat()
        
        document_cache[doc_id] = {
            "id": doc_id,
            "filename": filepath.name,
            "path": str(filepath),
            "content": filepath.read_text(),
            "size": stat.st_size,
            "modified": datetime.fromtimestamp(stat.st_mtime).isoformat()
        }
    
    print(f"Loaded {len(document_cache)} documents")

# Load documents at startup
load_documents()

if __name__ == "__main__":
    mcp.run()
'''

with open("document_assistant/server_step1.py", "w") as f:
    f.write(step1_code)

print("Step 1 complete: Basic server structure created")
print("\nThis creates:")
print("- MCP server instance")
print("- Document loading function")
print("- Document cache for fast access")

### Step 2: Add Search Tools

Add tools for searching and listing documents.

In [ ]:
step2_code = '''
"""Document Assistant MCP Server - Step 2: Search Tools"""

import os
import re
from pathlib import Path
from datetime import datetime
from typing import Optional
from mcp.server.fastmcp import FastMCP

# Configuration
DOCUMENTS_DIR = Path(__file__).parent / "documents"

# Create the MCP server
mcp = FastMCP(
    "Document Assistant",
    description="An MCP server for document management and analysis"
)

# Document storage
document_cache = {}

def load_documents():
    """Load all documents from the documents directory."""
    global document_cache
    document_cache = {}
    
    if not DOCUMENTS_DIR.exists():
        return
    
    for filepath in DOCUMENTS_DIR.glob("*.md"):
        doc_id = filepath.stem
        stat = filepath.stat()
        
        document_cache[doc_id] = {
            "id": doc_id,
            "filename": filepath.name,
            "path": str(filepath),
            "content": filepath.read_text(),
            "size": stat.st_size,
            "modified": datetime.fromtimestamp(stat.st_mtime).isoformat()
        }

load_documents()

# ============ TOOLS ============

@mcp.tool()
def list_documents() -> str:
    """List all available documents with their metadata.
    
    Returns:
        A formatted list of all documents with ID, filename, size, and modification date.
    """
    if not document_cache:
        return "No documents found."
    
    lines = ["Available Documents:", "=" * 50]
    
    for doc_id, doc in sorted(document_cache.items()):
        lines.append(f"\nID: {doc_id}")
        lines.append(f"  Filename: {doc['filename']}")
        lines.append(f"  Size: {doc['size']} bytes")
        lines.append(f"  Modified: {doc['modified']}")
    
    return "\n".join(lines)


@mcp.tool()
def search_documents(query: str, case_sensitive: bool = False) -> str:
    """Search for documents containing the specified text.
    
    Args:
        query: The text to search for in document contents
        case_sensitive: Whether the search should be case-sensitive (default: False)
    
    Returns:
        A list of matching documents with excerpts showing where the query was found.
    """
    if not query.strip():
        return "Error: Please provide a search query."
    
    results = []
    flags = 0 if case_sensitive else re.IGNORECASE
    
    for doc_id, doc in document_cache.items():
        content = doc["content"]
        matches = list(re.finditer(re.escape(query), content, flags))
        
        if matches:
            # Get excerpt around first match
            first_match = matches[0]
            start = max(0, first_match.start() - 50)
            end = min(len(content), first_match.end() + 50)
            excerpt = content[start:end].replace("\n", " ")
            
            if start > 0:
                excerpt = "..." + excerpt
            if end < len(content):
                excerpt = excerpt + "..."
            
            results.append({
                "doc_id": doc_id,
                "filename": doc["filename"],
                "match_count": len(matches),
                "excerpt": excerpt
            })
    
    if not results:
        return f"No documents found containing '{query}'."
    
    lines = [f"Found {len(results)} document(s) matching '{query}':", "=" * 50]
    
    for result in results:
        lines.append(f"\nDocument: {result['doc_id']}")
        lines.append(f"  Matches: {result['match_count']}")
        lines.append(f"  Excerpt: {result['excerpt']}")
    
    return "\n".join(lines)


@mcp.tool()
def get_document_stats() -> str:
    """Get statistics about the document collection.
    
    Returns:
        Statistics including total documents, total size, and average document size.
    """
    if not document_cache:
        return "No documents in the collection."
    
    total_docs = len(document_cache)
    total_size = sum(doc["size"] for doc in document_cache.values())
    total_words = sum(len(doc["content"].split()) for doc in document_cache.values())
    
    return f"""Document Collection Statistics:
{'=' * 40}
Total Documents: {total_docs}
Total Size: {total_size:,} bytes
Average Size: {total_size // total_docs:,} bytes
Total Words: {total_words:,}
Average Words per Document: {total_words // total_docs:,}
"""


if __name__ == "__main__":
    mcp.run()
'''

with open("document_assistant/server_step2.py", "w") as f:
    f.write(step2_code)

print("Step 2 complete: Search tools added")
print("\nNew tools:")
print("- list_documents(): List all available documents")
print("- search_documents(query): Search by content")
print("- get_document_stats(): Get collection statistics")

### Step 3: Add Resources

Add resources to expose document contents.

In [ ]:
step3_code = '''
"""Document Assistant MCP Server - Step 3: Resources"""

import os
import re
import json
from pathlib import Path
from datetime import datetime
from typing import Optional
from mcp.server.fastmcp import FastMCP

# Configuration
DOCUMENTS_DIR = Path(__file__).parent / "documents"

# Create the MCP server
mcp = FastMCP(
    "Document Assistant",
    description="An MCP server for document management and analysis"
)

# Document storage
document_cache = {}

def load_documents():
    """Load all documents from the documents directory."""
    global document_cache
    document_cache = {}
    
    if not DOCUMENTS_DIR.exists():
        return
    
    for filepath in DOCUMENTS_DIR.glob("*.md"):
        doc_id = filepath.stem
        stat = filepath.stat()
        
        document_cache[doc_id] = {
            "id": doc_id,
            "filename": filepath.name,
            "path": str(filepath),
            "content": filepath.read_text(),
            "size": stat.st_size,
            "modified": datetime.fromtimestamp(stat.st_mtime).isoformat()
        }

load_documents()

# ============ TOOLS ============

@mcp.tool()
def list_documents() -> str:
    """List all available documents with their metadata."""
    if not document_cache:
        return "No documents found."
    
    lines = ["Available Documents:", "=" * 50]
    for doc_id, doc in sorted(document_cache.items()):
        lines.append(f"\nID: {doc_id}")
        lines.append(f"  Filename: {doc['filename']}")
        lines.append(f"  Size: {doc['size']} bytes")
        lines.append(f"  Modified: {doc['modified']}")
    return "\n".join(lines)


@mcp.tool()
def search_documents(query: str, case_sensitive: bool = False) -> str:
    """Search for documents containing the specified text."""
    if not query.strip():
        return "Error: Please provide a search query."
    
    results = []
    flags = 0 if case_sensitive else re.IGNORECASE
    
    for doc_id, doc in document_cache.items():
        content = doc["content"]
        matches = list(re.finditer(re.escape(query), content, flags))
        
        if matches:
            first_match = matches[0]
            start = max(0, first_match.start() - 50)
            end = min(len(content), first_match.end() + 50)
            excerpt = content[start:end].replace("\n", " ")
            
            if start > 0:
                excerpt = "..." + excerpt
            if end < len(content):
                excerpt = excerpt + "..."
            
            results.append({
                "doc_id": doc_id,
                "filename": doc["filename"],
                "match_count": len(matches),
                "excerpt": excerpt
            })
    
    if not results:
        return f"No documents found containing '{query}'."
    
    lines = [f"Found {len(results)} document(s) matching '{query}':", "=" * 50]
    for result in results:
        lines.append(f"\nDocument: {result['doc_id']}")
        lines.append(f"  Matches: {result['match_count']}")
        lines.append(f"  Excerpt: {result['excerpt']}")
    return "\n".join(lines)


@mcp.tool()
def get_document_stats() -> str:
    """Get statistics about the document collection."""
    if not document_cache:
        return "No documents in the collection."
    
    total_docs = len(document_cache)
    total_size = sum(doc["size"] for doc in document_cache.values())
    total_words = sum(len(doc["content"].split()) for doc in document_cache.values())
    
    return f"""Document Collection Statistics:
{'=' * 40}
Total Documents: {total_docs}
Total Size: {total_size:,} bytes
Average Size: {total_size // total_docs:,} bytes
Total Words: {total_words:,}
Average Words per Document: {total_words // total_docs:,}
"""

# ============ RESOURCES ============

@mcp.resource("doc://{doc_id}")
def get_document(doc_id: str) -> str:
    """Get the full content of a specific document.
    
    Args:
        doc_id: The document identifier (filename without extension)
    
    Returns:
        The full content of the document
    """
    if doc_id not in document_cache:
        return f"Error: Document '{doc_id}' not found."
    
    return document_cache[doc_id]["content"]


@mcp.resource("docs://catalog")
def get_catalog() -> str:
    """Get the full document catalog as JSON.
    
    Returns:
        JSON-formatted catalog with all document metadata
    """
    catalog = []
    for doc_id, doc in document_cache.items():
        catalog.append({
            "id": doc["id"],
            "filename": doc["filename"],
            "size": doc["size"],
            "modified": doc["modified"],
            "word_count": len(doc["content"].split())
        })
    
    return json.dumps(catalog, indent=2)


@mcp.resource("doc://{doc_id}/metadata")
def get_document_metadata(doc_id: str) -> str:
    """Get metadata for a specific document.
    
    Args:
        doc_id: The document identifier
    
    Returns:
        JSON-formatted metadata for the document
    """
    if doc_id not in document_cache:
        return json.dumps({"error": f"Document '{doc_id}' not found."})
    
    doc = document_cache[doc_id]
    metadata = {
        "id": doc["id"],
        "filename": doc["filename"],
        "size": doc["size"],
        "modified": doc["modified"],
        "word_count": len(doc["content"].split()),
        "line_count": doc["content"].count("\n") + 1
    }
    
    return json.dumps(metadata, indent=2)


if __name__ == "__main__":
    mcp.run()
'''

with open("document_assistant/server_step3.py", "w") as f:
    f.write(step3_code)

print("Step 3 complete: Resources added")
print("\nNew resources:")
print("- doc://{doc_id} - Full document content")
print("- docs://catalog - Document catalog (JSON)")
print("- doc://{doc_id}/metadata - Document metadata (JSON)")

### Step 4: Add Prompts

Add prompts for common document analysis tasks.

In [ ]:
step4_code = '''
"""Document Assistant MCP Server - Step 4: Prompts (Complete Server)"""

import os
import re
import json
from pathlib import Path
from datetime import datetime
from typing import Optional
from mcp.server.fastmcp import FastMCP

# Configuration
DOCUMENTS_DIR = Path(__file__).parent / "documents"

# Create the MCP server
mcp = FastMCP(
    "Document Assistant",
    description="An MCP server for document management and analysis"
)

# Document storage
document_cache = {}

def load_documents():
    """Load all documents from the documents directory."""
    global document_cache
    document_cache = {}
    
    if not DOCUMENTS_DIR.exists():
        return
    
    for filepath in DOCUMENTS_DIR.glob("*.md"):
        doc_id = filepath.stem
        stat = filepath.stat()
        
        document_cache[doc_id] = {
            "id": doc_id,
            "filename": filepath.name,
            "path": str(filepath),
            "content": filepath.read_text(),
            "size": stat.st_size,
            "modified": datetime.fromtimestamp(stat.st_mtime).isoformat()
        }

load_documents()

# ============ TOOLS ============

@mcp.tool()
def list_documents() -> str:
    """List all available documents with their metadata."""
    if not document_cache:
        return "No documents found."
    
    lines = ["Available Documents:", "=" * 50]
    for doc_id, doc in sorted(document_cache.items()):
        lines.append(f"\nID: {doc_id}")
        lines.append(f"  Filename: {doc['filename']}")
        lines.append(f"  Size: {doc['size']} bytes")
        lines.append(f"  Modified: {doc['modified']}")
    return "\n".join(lines)


@mcp.tool()
def search_documents(query: str, case_sensitive: bool = False) -> str:
    """Search for documents containing the specified text."""
    if not query.strip():
        return "Error: Please provide a search query."
    
    results = []
    flags = 0 if case_sensitive else re.IGNORECASE
    
    for doc_id, doc in document_cache.items():
        content = doc["content"]
        matches = list(re.finditer(re.escape(query), content, flags))
        
        if matches:
            first_match = matches[0]
            start = max(0, first_match.start() - 50)
            end = min(len(content), first_match.end() + 50)
            excerpt = content[start:end].replace("\n", " ")
            
            if start > 0:
                excerpt = "..." + excerpt
            if end < len(content):
                excerpt = excerpt + "..."
            
            results.append({
                "doc_id": doc_id,
                "filename": doc["filename"],
                "match_count": len(matches),
                "excerpt": excerpt
            })
    
    if not results:
        return f"No documents found containing '{query}'."
    
    lines = [f"Found {len(results)} document(s) matching '{query}':", "=" * 50]
    for result in results:
        lines.append(f"\nDocument: {result['doc_id']}")
        lines.append(f"  Matches: {result['match_count']}")
        lines.append(f"  Excerpt: {result['excerpt']}")
    return "\n".join(lines)


@mcp.tool()
def get_document_stats() -> str:
    """Get statistics about the document collection."""
    if not document_cache:
        return "No documents in the collection."
    
    total_docs = len(document_cache)
    total_size = sum(doc["size"] for doc in document_cache.values())
    total_words = sum(len(doc["content"].split()) for doc in document_cache.values())
    
    return f"""Document Collection Statistics:
{'=' * 40}
Total Documents: {total_docs}
Total Size: {total_size:,} bytes
Average Size: {total_size // total_docs:,} bytes
Total Words: {total_words:,}
Average Words per Document: {total_words // total_docs:,}
"""

# ============ RESOURCES ============

@mcp.resource("doc://{doc_id}")
def get_document(doc_id: str) -> str:
    """Get the full content of a specific document."""
    if doc_id not in document_cache:
        return f"Error: Document '{doc_id}' not found."
    return document_cache[doc_id]["content"]


@mcp.resource("docs://catalog")
def get_catalog() -> str:
    """Get the full document catalog as JSON."""
    catalog = []
    for doc_id, doc in document_cache.items():
        catalog.append({
            "id": doc["id"],
            "filename": doc["filename"],
            "size": doc["size"],
            "modified": doc["modified"],
            "word_count": len(doc["content"].split())
        })
    return json.dumps(catalog, indent=2)


@mcp.resource("doc://{doc_id}/metadata")
def get_document_metadata(doc_id: str) -> str:
    """Get metadata for a specific document."""
    if doc_id not in document_cache:
        return json.dumps({"error": f"Document '{doc_id}' not found."})
    
    doc = document_cache[doc_id]
    metadata = {
        "id": doc["id"],
        "filename": doc["filename"],
        "size": doc["size"],
        "modified": doc["modified"],
        "word_count": len(doc["content"].split()),
        "line_count": doc["content"].count("\n") + 1
    }
    return json.dumps(metadata, indent=2)

# ============ PROMPTS ============

@mcp.prompt()
def summarize(doc_id: str) -> str:
    """Create a prompt to summarize a document.
    
    Args:
        doc_id: The document to summarize
    """
    if doc_id not in document_cache:
        return f"Error: Document '{doc_id}' not found."
    
    doc = document_cache[doc_id]
    return f"""Please summarize the following document:

Document: {doc['filename']}
---
{doc['content']}
---

Provide:
1. A brief summary (2-3 sentences)
2. Key points (bullet list)
3. Any action items or next steps mentioned"""


@mcp.prompt()
def compare(doc_id_1: str, doc_id_2: str) -> str:
    """Create a prompt to compare two documents.
    
    Args:
        doc_id_1: The first document to compare
        doc_id_2: The second document to compare
    """
    if doc_id_1 not in document_cache:
        return f"Error: Document '{doc_id_1}' not found."
    if doc_id_2 not in document_cache:
        return f"Error: Document '{doc_id_2}' not found."
    
    doc1 = document_cache[doc_id_1]
    doc2 = document_cache[doc_id_2]
    
    return f"""Please compare the following two documents:

=== Document 1: {doc1['filename']} ===
{doc1['content']}

=== Document 2: {doc2['filename']} ===
{doc2['content']}

Provide:
1. Main similarities between the documents
2. Key differences
3. How they might relate to each other
4. Any contradictions or inconsistencies"""


@mcp.prompt()
def extract_insights(doc_id: str, focus_area: str = "general") -> str:
    """Create a prompt to extract insights from a document.
    
    Args:
        doc_id: The document to analyze
        focus_area: The area to focus on (e.g., 'technical', 'business', 'security', 'general')
    """
    if doc_id not in document_cache:
        return f"Error: Document '{doc_id}' not found."
    
    doc = document_cache[doc_id]
    
    focus_instructions = {
        "general": "Extract any important insights, patterns, or notable information.",
        "technical": "Focus on technical details, architecture decisions, and implementation specifics.",
        "business": "Focus on business impact, ROI, timelines, and strategic implications.",
        "security": "Focus on security considerations, risks, compliance requirements, and vulnerabilities."
    }
    
    instruction = focus_instructions.get(focus_area, focus_instructions["general"])
    
    return f"""Please analyze the following document and extract key insights:

Document: {doc['filename']}
Focus Area: {focus_area}
---
{doc['content']}
---

{instruction}

Format your response as:
1. Key Insights (numbered list)
2. Recommendations based on the document
3. Questions that should be addressed
4. Related topics to explore"""


@mcp.prompt()
def generate_questions(doc_id: str) -> str:
    """Create a prompt to generate questions about a document.
    
    Args:
        doc_id: The document to generate questions for
    """
    if doc_id not in document_cache:
        return f"Error: Document '{doc_id}' not found."
    
    doc = document_cache[doc_id]
    
    return f"""Based on the following document, generate thoughtful questions that would help clarify or expand on the content:

Document: {doc['filename']}
---
{doc['content']}
---

Generate:
1. 5 clarifying questions (things that are unclear or need more detail)
2. 5 follow-up questions (things to explore further)
3. 3 devil's advocate questions (potential issues or challenges)"""


if __name__ == "__main__":
    mcp.run()
'''

# Save as the final server
with open("document_assistant/server.py", "w") as f:
    f.write(step4_code)

print("Step 4 complete: Prompts added - Server is complete!")
print("\nNew prompts:")
print("- summarize(doc_id) - Summarize a document")
print("- compare(doc_id_1, doc_id_2) - Compare two documents")
print("- extract_insights(doc_id, focus_area) - Extract insights with focus")
print("- generate_questions(doc_id) - Generate questions about a document")

## Part 4: Testing Your Server

Let's test the complete Document Assistant server.

### Testing with MCP Inspector

Run in terminal:
```bash
cd document_assistant
mcp dev server.py
```

Then test:
1. **Tools**: Call `list_documents()`, `search_documents("AI")`, `get_document_stats()`
2. **Resources**: Access `doc://project_proposal`, `docs://catalog`
3. **Prompts**: Use `summarize` with `doc_id="project_proposal"`

In [ ]:
# Test checklist

test_results = {
    "tools": {
        "list_documents": {
            "tested": False,
            "passed": False,
            "notes": ""
        },
        "search_documents": {
            "tested": False,
            "passed": False,
            "query_used": "",
            "results_count": 0,
            "notes": ""
        },
        "get_document_stats": {
            "tested": False,
            "passed": False,
            "notes": ""
        }
    },
    "resources": {
        "doc://project_proposal": {
            "tested": False,
            "passed": False,
            "notes": ""
        },
        "docs://catalog": {
            "tested": False,
            "passed": False,
            "notes": ""
        },
        "doc://project_proposal/metadata": {
            "tested": False,
            "passed": False,
            "notes": ""
        }
    },
    "prompts": {
        "summarize": {
            "tested": False,
            "passed": False,
            "notes": ""
        },
        "compare": {
            "tested": False,
            "passed": False,
            "notes": ""
        },
        "extract_insights": {
            "tested": False,
            "passed": False,
            "notes": ""
        }
    }
}

print("Test your server with MCP Inspector and fill in the results above!")
print("\nRun: cd document_assistant && mcp dev server.py")

## Part 5: VS Code Integration

Connect your Document Assistant to VS Code.

In [ ]:
import os

# Get the full path to the server
server_path = os.path.abspath("document_assistant/server.py")

print("Add this to your VS Code settings.json:")
print()
print('''{
  "github.copilot.chat.experimental.mcp.servers": {
    "document-assistant": {
      "command": "python",
      "args": ["''' + server_path + '''"]
    }
  }
}''')

### Testing in VS Code

After configuring, try these prompts in GitHub Copilot Chat:

1. "List all my documents"
2. "Search for documents about AI"
3. "Summarize the project proposal"
4. "Compare the project proposal with the technical spec"

In [ ]:
# Document your VS Code testing

vscode_tests = {
    "server_connected": False,
    "test_prompts": [
        {
            "prompt": "List all my documents",
            "tool_called": "",
            "result_quality": ""  # Good/Okay/Poor
        },
        {
            "prompt": "Search for documents about AI",
            "tool_called": "",
            "result_quality": ""
        },
        {
            "prompt": "Summarize the project proposal",
            "tool_called": "",
            "result_quality": ""
        }
    ],
    "overall_experience": ""
}

print("Document your VS Code integration experience above!")

## Part 6: Extension Challenges

Extend your Document Assistant with additional features.

### Challenge 1: Add Document Creation

Add a tool to create new documents.

In [ ]:
# TODO: Implement a create_document tool

create_document_code = '''
@mcp.tool()
def create_document(doc_id: str, content: str, overwrite: bool = False) -> str:
    """Create a new document in the collection.
    
    Args:
        doc_id: The identifier for the new document (will be used as filename)
        content: The content of the document
        overwrite: Whether to overwrite if document exists (default: False)
    
    Returns:
        Success message or error
    """
    # TODO: Implement this tool
    # 1. Check if document already exists (unless overwrite=True)
    # 2. Write the file to DOCUMENTS_DIR
    # 3. Reload the document cache
    # 4. Return success message
    pass
'''

print("Challenge 1: Implement the create_document tool")
print(create_document_code)

### Challenge 2: Add Tag Support

Add tagging capabilities to documents.

In [ ]:
# TODO: Implement tag support

tag_support_code = '''
# Store tags in a separate JSON file
TAGS_FILE = DOCUMENTS_DIR / "tags.json"

def load_tags() -> dict:
    """Load tags from the tags file."""
    if TAGS_FILE.exists():
        return json.loads(TAGS_FILE.read_text())
    return {}

def save_tags(tags: dict):
    """Save tags to the tags file."""
    TAGS_FILE.write_text(json.dumps(tags, indent=2))

@mcp.tool()
def add_tag(doc_id: str, tag: str) -> str:
    """Add a tag to a document.
    
    Args:
        doc_id: The document to tag
        tag: The tag to add
    """
    # TODO: Implement this
    pass

@mcp.tool()
def search_by_tag(tag: str) -> str:
    """Find all documents with a specific tag.
    
    Args:
        tag: The tag to search for
    """
    # TODO: Implement this
    pass

@mcp.tool()
def list_tags() -> str:
    """List all tags and their document counts."""
    # TODO: Implement this
    pass
'''

print("Challenge 2: Implement tag support")
print(tag_support_code)

### Challenge 3: Add Version History

Track document versions.

In [ ]:
# TODO: Implement version history

version_history_code = '''
# Store versions in a subdirectory
VERSIONS_DIR = DOCUMENTS_DIR / ".versions"

@mcp.tool()
def update_document(doc_id: str, new_content: str) -> str:
    """Update a document, saving the previous version.
    
    Args:
        doc_id: The document to update
        new_content: The new content
    """
    # TODO: Implement this
    # 1. Save current version to VERSIONS_DIR with timestamp
    # 2. Write new content to document
    # 3. Reload cache
    pass

@mcp.tool()
def list_versions(doc_id: str) -> str:
    """List all versions of a document.
    
    Args:
        doc_id: The document to list versions for
    """
    # TODO: Implement this
    pass

@mcp.resource("doc://{doc_id}/version/{version_id}")
def get_document_version(doc_id: str, version_id: str) -> str:
    """Get a specific version of a document.
    
    Args:
        doc_id: The document
        version_id: The version timestamp
    """
    # TODO: Implement this
    pass
'''

print("Challenge 3: Implement version history")
print(version_history_code)

## Part 7: Programmatic Access

Use the MCP Client SDK to access your Document Assistant programmatically.

In [ ]:
# Programmatic client for Document Assistant

client_code = '''
"""Programmatic client for the Document Assistant MCP server."""

import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def main():
    server_params = StdioServerParameters(
        command="python",
        args=["server.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize
            await session.initialize()
            
            # List all documents
            print("=" * 50)
            print("Listing documents...")
            result = await session.call_tool("list_documents", {})
            print(result.content[0].text)
            
            # Search for "AI"
            print("\n" + "=" * 50)
            print("Searching for 'AI'...")
            result = await session.call_tool("search_documents", {"query": "AI"})
            print(result.content[0].text)
            
            # Get document via resource
            print("\n" + "=" * 50)
            print("Reading project_proposal via resource...")
            resources = await session.list_resources()
            for resource in resources.resources:
                print(f"  Available: {resource.uri}")
            
            # Get collection stats
            print("\n" + "=" * 50)
            print("Getting statistics...")
            result = await session.call_tool("get_document_stats", {})
            print(result.content[0].text)

if __name__ == "__main__":
    asyncio.run(main())
'''

with open("document_assistant/client.py", "w") as f:
    f.write(client_code)

print("Client code saved to document_assistant/client.py")
print("\nRun with: cd document_assistant && python client.py")

## Part 8: Final Review

Let's review what you've built.

In [ ]:
# Final project summary

project_summary = {
    "server_name": "Document Assistant",
    "components": {
        "tools": [
            "list_documents() - List all documents",
            "search_documents(query) - Search by content",
            "get_document_stats() - Collection statistics"
        ],
        "resources": [
            "doc://{doc_id} - Document content",
            "docs://catalog - Full catalog (JSON)",
            "doc://{doc_id}/metadata - Document metadata"
        ],
        "prompts": [
            "summarize(doc_id) - Summarize document",
            "compare(doc_id_1, doc_id_2) - Compare documents",
            "extract_insights(doc_id, focus_area) - Extract insights",
            "generate_questions(doc_id) - Generate questions"
        ]
    },
    "files_created": [
        "document_assistant/server.py - Main server",
        "document_assistant/client.py - Programmatic client",
        "document_assistant/documents/*.md - Sample documents"
    ]
}

print("Document Assistant MCP Server")
print("=" * 50)
print("\nTools:")
for tool in project_summary["components"]["tools"]:
    print(f"  - {tool}")
print("\nResources:")
for resource in project_summary["components"]["resources"]:
    print(f"  - {resource}")
print("\nPrompts:")
for prompt in project_summary["components"]["prompts"]:
    print(f"  - {prompt}")

## Lab Summary

Congratulations! You've built a complete Document Assistant MCP server that demonstrates:

1. **Tools** for searching and listing documents
2. **Resources** for accessing document contents and metadata
3. **Prompts** for common document analysis tasks
4. **Integration** with VS Code and programmatic access

### Key Takeaways

- **Tools** are for actions (search, list, stats)
- **Resources** are for data access (read content, get metadata)
- **Prompts** are for reusable workflows (summarize, compare, analyze)
- Good docstrings are critical for tool discovery
- Test with MCP Inspector before integrating

### What's Next?

- Add your own document types and tools
- Implement the extension challenges
- Connect to real document storage (S3, database, etc.)
- Build more complex analysis prompts

### Workshop Complete!

You now have the skills to build custom MCP servers for any use case. The patterns you learned apply to:
- Database assistants
- API integrations
- Development tools
- Business applications

In [ ]:
# Final reflection

reflection = {
    "most_valuable_learning": "",
    "challenges_faced": "",
    "ideas_for_own_projects": "",
    "questions_remaining": ""
}

print("Reflect on your learning experience above!")
print("\nThank you for completing the MCP Workshop!")